In [5]:
from policyengine_uk import Microsimulation
from policyengine_uk.data import UKDataset

sim = Microsimulation(dataset=UKDataset("tmp.h5"))

In [11]:
sim.calculate("total_income", map_to="household")

               value  weight
0       6.676469e+06   750.0
1       2.114760e+06   871.0
2       5.184056e+06   710.0
3       3.132551e+06  1275.0
4       1.110000e+04  2457.0
...              ...     ...
100175  3.142957e+05     1.0
100176  1.049034e+06     1.0
100177  4.702986e+06     1.0
100178  1.095421e+06     1.0
100179  4.703790e+06     1.0

[100180 rows x 2 columns]

In [3]:
import plotly.express as px

px.scatter(
    dataset.person,
    x="age",
    y="capital_gains",
    opacity=0.1,
)

KeyboardInterrupt: 